In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

In [6]:
target_file_dir = "../datas/"
target_file_name = "real_estate_data_20211_20224_14_extract_by_cate"
target_file_end = ".csv"
df = pd.read_csv(target_file_dir + target_file_name + target_file_end)
df['TradePrice'] = df.TradePrice/10000

In [12]:
main_cols = [
    'Type','Prefecture', 'Municipality', 'DistrictName',
    'Structure', 'FloorPlan',
    'TradePrice', 'AgeAtTrade','Area',
    'TotalFloorArea','CoverageRatio', 'FloorAreaRatio',
    'BuildingYearW','TradeYear', 'TradeQuarter'
    ]

In [13]:
df2 = df.sort_values(['Prefecture','MunicipalityCode', 'DistrictName',
                     'BuildingYearW','TradePrice','Area']).copy()
df2['TradeNo'] = range(len(df2))
df2_main = df2.loc[:,['TradeNo']+main_cols]
df2_sub =  df2.drop(main_cols, axis=1)

In [15]:
print(df2.shape, df2_main.shape, df2_sub.shape)

(29548, 33) (29548, 16) (29548, 18)
